# Grid to Station


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install scikit-gstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 688.0/688.0 KB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-gstat: filename=scikit_gstat-1.0.9-py3-none-any.whl size=702914 sha256=e3e9bd39c3325449be50e573a38aa3fb2f3db0604ca72c6397cb727b96e2f46d
  Stored in directory: /root/.cache/pip/wheels/16/b2/bd/9bfa24f6e4859128d54ad8fbb1ed3dcb66264c91750ae92f0f
Successfully built scikit-gstat


# Final


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import skgstat as skg

In [ ]:
# Importing the csv data
df = pd.read_csv("/content/drive/MyDrive/Krig values/1996_rainfall_kriging.csv")                                              

# Importing the Station data
station = pd.read_csv("/content/drive/MyDrive/Krig values/rf40_station.csv")


gcoords = []                # Grid coordinates
mdatabase = pd.DataFrame()  # Final database to contain the values  

# Loop used in case of multiple years 
for y in {96}:                                                                                                                        

  # 31 days months 
  for month in {1,3,5,7,8,10,12}:      
          
      for day in range(1,32): 

          dfm = df[df['Month'] == month]
          dfm = dfm[dfm['Day'] == day]
          value = dfm['Rainfall'].values

          # Going from grid to station
          # Defining Lat Lon of Grid points
          
          lat = list(dfm['Latitude']) 
          lon = list(dfm['Longitude'])
          gcoords = [[lat[i],lon[i]] for i in range(len(dfm))]

          #Station values for 1995
          sf = station.loc[ (station['Year'] == y) & (station['Month'] == month)]

          #Defining Lat Lon of stations
          ulat = (sf['Latitude'].values)
          ulon = (sf['Longitude'].values)
          coords = [[ulat[i],ulon[i]] for i in range(len(ulat))]


          X = [i[0] for i in coords]
          Y = [j[1] for j in coords] 
          
          
          #Actual Rainfall
          arain = list(station[str(day)].loc[ (station['Year'] == y) & (station['Month'] == month)])

          try :
            if np.all(value == 0):
              newdata = pd.DataFrame({'Latitude':X,'Longitude': Y, 'Year': [dfm['Year'].values[0]]*len(X), 'Month' : month , 'Day' : day, 'Predicted Rainfall' : [0]*len(X), 'Actual Rainfall': arain})  # No district or station because these are the grid values. 
              mdatabase = pd.concat([mdatabase,newdata])
              print("year: ",y,"month: ",month,",day: ",day,": ", Surf,len(Surf))
              continue
            V = skg.Variogram(gcoords,value) 


          except:
            Surf = [0]*len(X)
            newdata = pd.DataFrame({'Latitude':X,'Longitude': Y, 'Year': [dfm['Year'].values[0]]*len(X), 'Month' : month , 'Day' : day, 'Predicted Rainfall' : [0]*len(X), 'Actual Rainfall': arain})  # No district or station because these are the grid values. 
            mdatabase = pd.concat([mdatabase,newdata])
            print("exception","year: ",y,"month: ",month,",day: ",day,": ", len(Surf))
            continue

          ok = skg.OrdinaryKriging(V , min_points = 1, max_points = 5, mode = 'exact')
    
          Surf = ok.transform(X,Y)


          # Removing the negative values attained in Kriging
          for i in range(len(Surf)):
            if Surf[i] < 0.0:
              Surf[i] = 0


          newdata = pd.DataFrame({'Latitude':X,'Longitude': Y, 'Year': [dfm['Year'].values[0]]*len(X), 'Month' : month , 'Day' : day, 'Predicted Rainfall' : Surf, 'Actual Rainfall' : arain})  # No district or station because these are the grid values. 
          mdatabase = pd.concat([mdatabase,newdata])
          temp = pd.DataFrame({'Latitude':X,'Longitude': Y, 'Year': [dfm['Year'].values[0]]*len(X), 'Month' : month , 'Day' : day, 'Predicted Rainfall' : Surf, 'Actual Rainfall' : arain})
          print("year: ",y,"month: ",month,",day: ",day,": ", Surf,len(Surf))

  # 30 days month
  for month in {4,6,9,11}:      
          
      for day in range(1,31): 

          dfm = df[df['Month'] == month]
          dfm = dfm[dfm['Day'] == day]
          value = dfm['Rainfall'].values

          # Going from grid to station
          # Defining Lat Lon of Grid points
          
          lat = list(dfm['Latitude']) 
          lon = list(dfm['Longitude'])
          gcoords = [[lat[i],lon[i]] for i in range(len(dfm))]

          #Station values for 1995
          sf = station.loc[(station['Year'] == y) & (station['Month'] == month)]

          #Defining Lat Lon of stations
          ulat = (sf['Latitude'].values)
          ulon = (sf['Longitude'].values)
          coords = [[ulat[i],ulon[i]] for i in range(len(ulat))]

          X = [i[0] for i in coords]
          Y = [j[1] for j in coords] 
          
          #Actual Rainfall
          arain = list(station[str(day)].loc[ (station['Year'] == y) & (station['Month'] == month)])

          try :
            if np.all(value == 0):
              newdata = pd.DataFrame({'Latitude':X,'Longitude': Y, 'Year': [dfm['Year'].values[0]]*len(X), 'Month' : month , 'Day' : day, 'Predicted Rainfall' : [0]*len(X), 'Actual Rainfall': arain})  # No district or station because these are the grid values. 
              mdatabase = pd.concat([mdatabase,newdata])
              print("year: ",y,"month: ",month,",day: ",day,": ", Surf,len(Surf))
              continue
            V = skg.Variogram(gcoords,value) 
            #V = skg.Variogram(gcoords,value, fit_method = 'lm')


          except:
            #if np.all(value == 0):
            Surf = [0]*len(X)
            newdata = pd.DataFrame({'Latitude':X,'Longitude': Y, 'Year': [dfm['Year'].values[0]]*len(X), 'Month' : month , 'Day' : day, 'Predicted Rainfall' : [0]*len(X), 'Actual Rainfall': arain})  # No district or station because these are the grid values. 
            mdatabase = pd.concat([mdatabase,newdata])
            print("exception","year: ",y,"month: ",month,",day: ",day,": ", len(Surf))
            continue
            #V = skg.Variogram(gcoords,value, fit_method = 'lm')

          ok = skg.OrdinaryKriging(V , min_points = 1, max_points = 5, mode = 'exact')
    
          Surf = ok.transform(X,Y)
          
          # Removing the negative values attained in Kriging
          for i in range(len(Surf)):
            if Surf[i] < 0.0:
              Surf[i] = 0

          newdata = pd.DataFrame({'Latitude':X,'Longitude': Y, 'Year': [dfm['Year'].values[0]]*len(X), 'Month' : month , 'Day' : day, 'Predicted Rainfall' : Surf, 'Actual Rainfall' : arain})  # No district or station because these are the grid values. 
          mdatabase = pd.concat([mdatabase,newdata])
          temp = pd.DataFrame({'Latitude':X,'Longitude': Y, 'Year': [dfm['Year'].values[0]]*len(X), 'Month' : month , 'Day' : day, 'Predicted Rainfall' : Surf, 'Actual Rainfall' : arain})
          print("year: ",y,"month: ",month,",day: ",day,": ", Surf,len(Surf))

  # 27 days month
  for month in {2}:       
          
      for day in range(1,30):                                                                                       #Change Day here

          dfm = df[df['Month'] == month]
          dfm = dfm[dfm['Day'] == day]
          value = dfm['Rainfall'].values

          # Going from grid to station
          # Defining Lat Lon of Grid points
          
          lat = list(dfm['Latitude']) 
          lon = list(dfm['Longitude'])
          gcoords = [[lat[i],lon[i]] for i in range(len(dfm))]

          #Station values for 1995
          sf = station.loc[ (station['Year'] == y) & (station['Month'] == month)]

          #Defining Lat Lon of stations
          ulat = (sf['Latitude'].values)
          ulon = (sf['Longitude'].values)
          coords = [[ulat[i],ulon[i]] for i in range(len(ulat))]


          X = [i[0] for i in coords]
          Y = [j[1] for j in coords] 
          #Surf = ok.transform(X,Y)
          
          
          #Actual Rainfall
          arain = list(station[str(day)].loc[ (station['Year'] == y) & (station['Month'] == month)])

          try :
            if np.all(value == 0):
              newdata = pd.DataFrame({'Latitude':X,'Longitude': Y, 'Year': [dfm['Year'].values[0]]*len(X), 'Month' : month , 'Day' : day, 'Predicted Rainfall' : [0]*len(X), 'Actual Rainfall': arain})  # No district or station because these are the grid values. 
              mdatabase = pd.concat([mdatabase,newdata])
              print("year: ",y,"month: ",month,",day: ",day,": ", Surf,len(Surf))
              continue
            V = skg.Variogram(gcoords,value) 


          except:
            Surf = [0]*len(X)
            newdata = pd.DataFrame({'Latitude':X,'Longitude': Y, 'Year': [dfm['Year'].values[0]]*len(X), 'Month' : month , 'Day' : day, 'Predicted Rainfall' : [0]*len(X), 'Actual Rainfall': arain})  # No district or station because these are the grid values. 
            mdatabase = pd.concat([mdatabase,newdata])
            print("exception","year: ",y,"month: ",month,",day: ",day,": ", len(Surf))
            continue

          ok = skg.OrdinaryKriging(V , min_points = 1, max_points = 5, mode = 'exact')
    
          Surf = ok.transform(X,Y)
          
          # Removing the negative values attained in Kriging
          for i in range(len(Surf)):
            if Surf[i] < 0.0:
              Surf[i] = 0

          newdata = pd.DataFrame({'Latitude':X,'Longitude': Y, 'Year': [dfm['Year'].values[0]]*len(X), 'Month' : month , 'Day' : day, 'Predicted Rainfall' : Surf, 'Actual Rainfall' : arain})  # No district or station because these are the grid values. 
          mdatabase = pd.concat([mdatabase,newdata])
          temp = pd.DataFrame({'Latitude':X,'Longitude': Y, 'Year': [dfm['Year'].values[0]]*len(X), 'Month' : month , 'Day' : day, 'Predicted Rainfall' : Surf, 'Actual Rainfall' : arain})
          print("year: ",y,"month: ",month,",day: ",day,": ", Surf,len(Surf))

  mdatabase.to_csv("/output/1996_output.csv")                       
  mdatabase = mdatabase.iloc[0:0]               #making all years 0 for the next iteration of year 

exception year:  96 month:  1 ,day:  1 :  4930
exception year:  96 month:  1 ,day:  2 :  4930
exception year:  96 month:  1 ,day:  3 :  4930
exception year:  96 month:  1 ,day:  4 :  4930
exception year:  96 month:  1 ,day:  5 :  4930
exception year:  96 month:  1 ,day:  6 :  4930
year:  96 month:  1 ,day:  7 :  [0. 0. 0. ... 0. 0. 0.] 4930
